# 混合式编程

In [7]:
import sys
sys.path.append('..')
import torch
from torch import nn
import d2l
import numpy

In [8]:
# 生产网络的工厂模式 
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[0.0067, 0.1488]], grad_fn=<AddmmBackward0>)

通过使用torch.jit.script函数来转换模型

In [9]:
net = torch.jit.script(net)
net(x)

tensor([[0.0067, 0.1488]], grad_fn=<DifferentiableGraphBackward>)

In [10]:
class Benchmark:
    """用于测量运行时间"""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self
    
    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [11]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): 
        net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'): 
    for i in range(1000): net(x)

无torchscript: 0.0240 sec
有torchscript: 0.0192 sec


In [12]:
net.save('my_mlp')

### 通过后端异步处理

In [14]:
# GPU计算热身
device = d2l.try_gpu()

a = torch.randn(size=(1000, 1000), device=device) 
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.4492 sec
torch: 0.0023 sec


In [17]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.mps.synchronize()

Done: 0.0250 sec


- PyTorch有一个用于与用户直接交互的前端(例如通过Python)
- 还有一个由系统用来执行计算的后端。

![threading.svg](https://zh.d2l.ai/_images/threading.svg)

前端和后端的交互

每当Python前端线程执行前三条语句中的一条语句时，它只是将任务返回到后端队列。当最后一个语句的结果需要被打印出来时，Python前端线程将等待C++后端线程完成变量z的结果计算。

In [18]:
x = torch.ones((1, 2), device=device) 
y = torch.ones((1, 2), device=device)
z=x*y+2
z

tensor([[3., 3.]], device='mps:0')

Python前端线程和C++后端线程之间的简化交互可以概括如下：

1. 前端命令后端将计算任务y = x + 1插入队列；
2. 然后后端从队列接收计算任务并执行；
3. 然后后端将计算结果返回到前端。